### This notebook demonstrates the ability of the DisparateImpactRemover algorithm.
The algorithm corrects for imbalanced selection rates between unprivileged and privileged groups at various levels of repair. It follows the guidelines set forth by [1] for training the algorithm and classifier and uses the AdultDataset as an example.

In [19]:
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function
from __future__ import unicode_literals

from matplotlib import pyplot as plt

import sys
sys.path.append("../")
import warnings

import numpy as np
from tqdm import tqdm

from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import f1_score


from aif360.algorithms.preprocessing import DisparateImpactRemover
from aif360.datasets import AdultDataset
from aif360.metrics import BinaryLabelDatasetMetric

In [20]:
protected = 'sex'
ad = AdultDataset(protected_attribute_names=[protected],
    privileged_classes=[['Male']], categorical_features=[],
    features_to_keep=['age', 'education-num', 'capital-gain', 'capital-loss', 'hours-per-week']) # OKEY
    # just numerical

In [21]:
scaler = MinMaxScaler(copy=False)

In [22]:
test, train = ad.split([16281]) # ? -> não sei
train.features = scaler.fit_transform(train.features)
test.features = scaler.fit_transform(test.features)

index = train.feature_names.index(protected)

In [23]:
DIs = []
SPD = []
F1s = []
for level in tqdm(np.linspace(0., 1., 11)):
    di = DisparateImpactRemover(repair_level=level)
    train_repd = di.fit_transform(train)
    test_repd = di.fit_transform(test)
    
    X_tr = np.delete(train_repd.features, index, axis=1)
    X_te = np.delete(test_repd.features, index, axis=1)
    y_tr = train_repd.labels.ravel()
    
    lmod = LogisticRegression(class_weight='balanced', solver='liblinear')
    lmod.fit(X_tr, y_tr)
    
    test_repd_pred = test_repd.copy()
    test_repd_pred.labels = lmod.predict(X_te)

    p = [{protected: 1}]
    u = [{protected: 0}]
    cm = BinaryLabelDatasetMetric(test_repd_pred, privileged_groups=p, unprivileged_groups=u)
    y_true = test_repd.labels.ravel()
    y_pred = test_repd_pred.labels.ravel()
    DIs.append(cm.disparate_impact())
    SPD.append(cm.statistical_parity_difference())
    F1s.append(f1_score(y_true, y_pred))


100%|██████████| 11/11 [01:13<00:00,  6.67s/it]


In [31]:
i = 0
file_name = ".//results_DI_remover//adult_results_DI_remover.csv"
with open(file_name, 'a') as files:
    for level in (np.linspace(0., 1., 11)):
        print("Nível de Reparo: ", level, file = files)
        print("Statistical Parity: ", SPD[i],  file = files)
        print("F1-Score: ", F1s[i],  file = files)
        i+=1

FileNotFoundError: [Errno 2] No such file or directory: './/results_DI_remover//adult_results_DI_remover.csv'

    References:
        .. [1] M. Feldman, S. A. Friedler, J. Moeller, C. Scheidegger, and
           S. Venkatasubramanian, "Certifying and removing disparate impact."
           ACM SIGKDD International Conference on Knowledge Discovery and Data
           Mining, 2015.